<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Storage Test</span> </p>

In [0]:
#!import "../Import/Importers"
#!import "TestData"

# Initialization 

In [0]:
static HashSet<string> myimportscopesatinception;
static HashSet<string> myimportscopesnotatinception;
static HashSet<string> myprimaryscope;
static HashSet<string> mysecondaryscope;
static HashSet<string> myprimaryscopefromparsed;
static HashSet<string> myprimaryscopefromlinked;

public class ImportStorage
{
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;
    
    // Constants
    private static int periodicityInMonths => 3; // Revisit
    
    //Format
    public string ImportFormat => args.ImportFormat; 
    
    //Time Periods 
    public (int Year, int Month) CurrentReportingPeriod => (args.Year, args.Month);
    public (int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic

    //Partitions
    public Guid PartitionByRn;
    public Guid TargetPartition;
    public Guid DefaultPartition;
    public Guid PreviousPeriodPartition;    

    //Projections
    private ProjectionConfiguration[] ProjectionConfiguration;
    
    //DataNodes
    public IDictionary<string, DataNodeData> DataNodeDataBySystemName { get; private set; }
    public IDictionary<ImportScope, HashSet<string>> DataNodesByImportScope { get; private set; }
    public IDictionary<string, ICollection<int?>> AccidentYearsByDataNode { get; private set; }
    //Variables
    public IDictionary<string, ICollection<RawVariable>> RawVariablesByImportIdentity { get; private set; }
    public IDictionary<string, ICollection<IfrsVariable>> IfrsVariablesByImportIdentity { get; private set; }
        
    //Parameters
    public Dictionary<string, YieldCurve> LockedInYieldCurve { get; private set; }
    public Dictionary<string, Dictionary<int, YieldCurve>> CurrentYieldCurve { get; private set; }
    public Dictionary<string, Dictionary<int, PartnerRating>> PartnerRating { get; private set; }
    public Dictionary<string, Dictionary<int, CreditDefaultRate>> CreditDefaultRates { get; private set; }
    public Dictionary<string, Dictionary<int, SingleDataNodeParameter>> SingleDataNodeParametersByGoc { get; private set; }
    public Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>> InterDataNodeParametersByGoc { get; private set; }
    public Dictionary<AocStep, AocConfiguration> AocConfigurationByAocStep { get; private set; }
    
    private Dictionary<InputSource, HashSet<AocStep>> aocStepByInputSource;
    
    //Dimensions
    public Dictionary<string, AmountType> AmountTypeDimension { get; private set; }
    public Dictionary<string, Novelty> NoveltyDimension { get; private set; }
    public Dictionary<string, EstimateType> EstimateTypeDimension { get; private set; }
    public Dictionary<string, HashSet<string>> EstimateTypesByImportFormat { get; private set; }
    public Dictionary<string, string> ExperienceAdjustEstimateTypeMapping { get; private set; } 
    public HashSet<string> ImportActualEstimateTypes => GetImportActualEstimateType();

    //Constructor
    public ImportStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    //Initialize
    public async Task InitializeAsync()
    {   
        //Dimensions
        var estimateTypes = await workspace.Query<EstimateType>().ToArrayAsync();
        
        EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}}; //TODO move this logic
        
        //Hierarchy Cache
        await hierarchyCache.InitializeAsync<AmountType>();
        
        //EstimateType to load and to update
        EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow }
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % periodicityInMonths == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        PartitionByRn = (await querySource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        TargetPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        DefaultPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Set Partitions
        await querySource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
        await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        //Get data from Workspace (result of parsing)
        var parsedRawVariables = await workspace.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, ImportFormat);
        var parsedIfrsVariables = await workspace.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, ImportFormat);
        
        //DataNodes
        DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);
        
        //Accident Years
        AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
            (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
            .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
        // Import Scopes and Data Node relationship parameters
        InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
        
        var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
        var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();

        var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
        var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
        var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
        DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
        // Parameters
        PartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
        CreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
        SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);
        LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
        CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
        AocConfigurationByAocStep = await querySource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
        aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
        //Previous Period
        var openingRawVariables = Enumerable.Empty<RawVariable>();
        var openingIfrsVariables = Enumerable.Empty<IfrsVariable>();

        var allImportScopesAtInceptionYear = allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).Where(dnd => dnd.Year == args.Year).Select(x => x.DataNode).ToHashSet();
        var allImportScopesNotAtInceptionYear = allImportScopes.Except(allImportScopesAtInceptionYear).ToHashSet();
myimportscopesatinception=allImportScopesAtInceptionYear;
myimportscopesnotatinception=allImportScopesNotAtInceptionYear;
myprimaryscope = primaryScope;
mysecondaryscope = secondaryScope;
myprimaryscopefromparsed = primaryScopeFromParsedVariables;
myprimaryscopefromlinked = primaryScopeFromLinkedReinsurance;
        if(allImportScopesAtInceptionYear.Any()) {
            var primaryScopeAtInceptionYear = allImportScopesAtInceptionYear.Where(dn => primaryScope.Contains(dn));
            var secondaryScopeAtInceptionYear = allImportScopesAtInceptionYear.Where(dn => secondaryScope.Contains(dn));

            openingIfrsVariables = await querySource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == TargetPartition && iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I)
                                    .Where(iv => primaryScopeAtInceptionYear.Contains(iv.DataNode) && ImportFormat != ImportFormats.Opening
                                                 && EstimateTypesByImportFormat[InputSource.Opening.ToString()].Contains(iv.EstimateType) 
                                                 || secondaryScopeAtInceptionYear.Contains(iv.DataNode)).ToArrayAsync();
        }

        if(allImportScopesNotAtInceptionYear.Any()) {
            PreviousPeriodPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>()
                                        .Where(p => p.ReportingNode == args.ReportingNode && p.Year == PreviousReportingPeriod.Year 
                                                 && p.Month == PreviousReportingPeriod.Month && p.Scenario == null).ToArrayAsync()).Single().Id;
            
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(PreviousPeriodPartition);
        
            //Perform queries to previous Period
            openingRawVariables = (await querySource.Query<RawVariable>()
                                   .Where(rv => rv.Partition == PreviousPeriodPartition && rv.AocType == AocTypes.CL)
                                   .Where(v => primaryScope.Contains(v.DataNode)).ToArrayAsync())
                                   .Select(rv => rv with {AocType = AocTypes.BOP, Novelty = Novelties.I, 
                                                 Values = rv.Values.Skip(MonthInAYear).ToArray(), Partition = TargetPartition});
            
            openingIfrsVariables = openingIfrsVariables.Union((await querySource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == PreviousPeriodPartition && iv.AocType == AocTypes.EOP)
                                    .Where(v => allImportScopesNotAtInceptionYear.Contains(v.DataNode)).ToArrayAsync())
                                    .Select(iv => iv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Partition = TargetPartition}),
                                    EqualityComparer<IfrsVariable>.Instance);
            
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);

            // TODO: print error if 
            //openingRawVariables.Select(x => x.DataNode).ToHashSet() != dataNodesWithPreviousPeriod
        }
        
        //Variables
        var rawVariables = parsedRawVariables.Concat(openingRawVariables)
                                             .Concat(await querySource.Query<RawVariable>()
                                                                      .Where(rv => rv.Partition == TargetPartition)
                                                                      .Where(rv => primaryScopeFromLinkedReinsurance.Contains(rv.DataNode)).ToArrayAsync()); 
        
        var ifrsVariables = parsedIfrsVariables.Union(openingIfrsVariables, EqualityComparer<IfrsVariable>.Instance)
                                               .Concat(await querySource.Query<IfrsVariable>()
                                                                        .Where(iv => iv.Partition == TargetPartition && !(iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I))
                                                                        .Where(iv => primaryScopeFromParsedVariables.Contains(iv.DataNode) 
                                                                                     && !EstimateTypesByImportFormat[ImportFormat].Contains(iv.EstimateType)  
                                                                                     || primaryScopeFromLinkedReinsurance.Contains(iv.DataNode) 
                                                                                     || secondaryScope.Contains(iv.DataNode)).ToArrayAsync());

        if(DefaultPartition != TargetPartition) {
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(DefaultPartition);
            var defaultRawVariables = await querySource.Query<RawVariable>().Where(rv => rv.Partition == DefaultPartition && primaryScope.Contains(rv.DataNode)).ToArrayAsync();
            var defaultIfrsVariables = await querySource.Query<IfrsVariable>().Where(iv => iv.Partition == DefaultPartition && allImportScopes.Contains(iv.DataNode)).ToArrayAsync();         
            rawVariables = rawVariables.Union(defaultRawVariables, EqualityComparer<RawVariable>.Instance);
            ifrsVariables = ifrsVariables.Union(defaultIfrsVariables, EqualityComparer<IfrsVariable>.Instance);
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        }

        RawVariablesByImportIdentity = (IDictionary<string, ICollection<RawVariable>>)rawVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<RawVariable>)v.ToArray());
        IfrsVariablesByImportIdentity = (IDictionary<string, ICollection<IfrsVariable>>)ifrsVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<IfrsVariable>)v.ToArray());
    }
    
    //Getters
    
    //Periods
    public ValuationPeriod GetValuationPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].ValuationPeriod;
    public PeriodType GetYieldCurvePeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].YcPeriod;
    public PeriodType GetCreditDefaultRiskPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].CdrPeriod;
    
    public IEnumerable<AocStep> GetAllAocSteps(InputSource source) => aocStepByInputSource[source];
    public IEnumerable<AocStep> GetCalculatedTelescopicAocSteps() => AocConfigurationByAocStep.Where(kvp => kvp.Value.DataType == DataType.CalculatedTelescopic).Select(kvp => kvp.Key);

    //YieldCurve
    public double[] GetYearlyYieldCurve(ImportIdentity id, string economicBasis) {
        var yc = GetYieldCurve(id, economicBasis);
        return yc.Values.Skip(args.Year - yc.Year).ToArray(); //Check if the returned array is empty? Log Warning?
    }
    
    public YieldCurve GetYieldCurve(ImportIdentity id, string economicBasis) => (economicBasis, GetYieldCurvePeriod(id)) switch {
            (EconomicBases.C, PeriodType.BeginningOfPeriod ) => GetShift(id.ProjectionPeriod) > 0 
                                     ? CurrentYieldCurve[id.DataNode][CurrentPeriod]
                                     : CurrentYieldCurve[id.DataNode][PreviousPeriod],
            (EconomicBases.C, PeriodType.EndOfPeriod) => CurrentYieldCurve[id.DataNode][CurrentPeriod],            
            (EconomicBases.L, _ ) => LockedInYieldCurve[id.DataNode],
            (_, PeriodType.NotApplicable) => (YieldCurve)ApplicationMessage.Log(Error.YieldCurvePeriodNotApplicable, id.AocType, id.Novelty),
            (_, _) => (YieldCurve)ApplicationMessage.Log(Error.EconomicBasisNotFound, id.DataNode)
       };

    //int Identity.ProjectionPeriod 
    public int GetProjectionCount() => ProjectionConfiguration.Count();
    public int GetShift(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].Shift;
    public int GetTimeStep(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].TimeStep;
    
    public PeriodType GetPeriodType(string amountType, string estimateType) => 
                    amountType != null && AmountTypeDimension.TryGetValue(amountType, out var at) 
                                ? at.PeriodType
                                : estimateType != null && EstimateTypeDimension.TryGetValue(estimateType, out var ct) 
                                    ? ct.PeriodType : PeriodType.EndOfPeriod;

    //Variables and Cash flows
    
    public IEnumerable<RawVariable> GetRawVariables(string dataNode) => RawVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<RawVariable>();
    public IEnumerable<IfrsVariable> GetIfrsVariables(string dataNode) => IfrsVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<IfrsVariable>();
    
    public double[] GetValues(ImportIdentity id, Func<RawVariable, bool> whereClause) => GetRawVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Aggregate()?.Values ?? Enumerable.Empty<double>().ToArray();
    public double GetValue(ImportIdentity id, Func<IfrsVariable, bool> whereClause) => GetIfrsVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Aggregate()?.Value ?? 0;
    public double[] GetValues(ImportIdentity id, string amountType, string estimateType, int? accidentYear) => GetValues(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType);

    public double GetValue(ImportIdentity id, string amountType, string estimateType, int? accidentYear) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType);
    public double GetValue(ImportIdentity id, string amountType, string estimateType, string economicBasis, int? accidentYear) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType && v.EconomicBasis == economicBasis);
   
    //Novelty
    private IEnumerable<string> GetNoveltiesForAocType(string aocType, IEnumerable<AocStep> aocConfiguration) => aocConfiguration.Where(aocStep => aocStep.AocType == aocType).Select(aocStep => aocStep.Novelty);
    public IEnumerable<string> GetNovelties() => NoveltyDimension.Keys;
    public IEnumerable<string> GetNovelties(string aocType) => GetNoveltiesForAocType(aocType, AocConfigurationByAocStep.Keys);
    public IEnumerable<string> GetNovelties(string aocType, InputSource inputSource) => GetNoveltiesForAocType(aocType, aocStepByInputSource[inputSource]);
    
    //Accident years
    public IEnumerable<int?> GetAccidentYears(string dataNode) => AccidentYearsByDataNode.TryGetValue(dataNode, out var accidentYear) ? accidentYear : new int?[] { null };
    
    // Parameters
    public double GetNonPerformanceRiskRate (ImportIdentity identity) {
        var period = GetCreditDefaultRiskPeriod(identity) == PeriodType.BeginningOfPeriod ? PreviousPeriod : CurrentPeriod;
       
        if(!DataNodeDataBySystemName.TryGetValue(identity.DataNode, out var dataNodeData))     ApplicationMessage.Log(Error.DataNodeNotFound, identity.DataNode);
        if(dataNodeData.Partner == null)                                                       ApplicationMessage.Log(Error.PartnerNotFound, identity.DataNode);
        // if Partner == Internal then return 0;
        if(!PartnerRating.TryGetValue(dataNodeData.Partner, out var rating))                   ApplicationMessage.Log(Error.RatingNotFound, dataNodeData.Partner);
        if(!CreditDefaultRates.TryGetValue(rating[period].CreditRiskRating, out var rate))     ApplicationMessage.Log(Error.CreditDefaultRateNotFound, rating[period].CreditRiskRating);
        return Math.Pow(1d + rate[period].Values[0], 1d / 12d) - 1d;
    }
    
    public double GetPremiumAllocationFactor(ImportIdentity id) => SingleDataNodeParametersByGoc.TryGetValue(id.DataNode, out var singleDataNodeParameter) 
                                                                                ? singleDataNodeParameter[CurrentPeriod].PremiumAllocation : DefaultPremiumExperienceAdjustmentFactor;
    
    // Data Node relationships
    public IEnumerable<string> GetUnderlyingGic(ImportIdentity id) => !InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
        ? Enumerable.Empty<string>()
        : interDataNodeParameters[CurrentPeriod].Select(x => x.DataNode != id.DataNode ? x.DataNode : x.LinkedDataNode).Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance);
    
    public double GetReinsuranceCoverage (ImportIdentity id, string gic)  
    {
        var targetPeriod = AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].RcPeriod == PeriodType.EndOfPeriod ? CurrentPeriod : PreviousPeriod;
        return InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
            ? interDataNodeParameters[targetPeriod].FirstOrDefault(x => x.DataNode == gic || x.LinkedDataNode == gic).ReinsuranceCoverage
            : (double)ApplicationMessage.Log(Error.ReinsuranceCoverage, id.DataNode);
    }
    
    // Import Scope
    public bool IsPrimaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Primary].Contains(dataNode);
    public bool IsSecondaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Secondary].Contains(dataNode);
    
    // Other
    public IEnumerable<string> GetNonAttributableAmountType() => new string[]{AmountTypes.NE};
    public IEnumerable<string> GetAttributableExpenseAndCommissionAmountType() => hierarchyCache.Get<AmountType>(AmountTypes.ACA).Descendants(includeSelf : true).Select(x => x.SystemName)
                                                                                   .Concat(hierarchyCache.Get<AmountType>(AmountTypes.AEA).Descendants(includeSelf : true).Select(x => x.SystemName));
    public IEnumerable<string> GetInvestmentClaims() => hierarchyCache.Get<AmountType>(AmountTypes.ICO).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetPremiums() => hierarchyCache.Get<AmountType>(AmountTypes.PR).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetClaims() => hierarchyCache.Get<AmountType>(AmountTypes.CL).Descendants().Select(x => x.SystemName);
}

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());
Workspace.Reset(x => x.ResetCurrentPartitions());

In [0]:
await Import.FromString(novelties).WithType<Novelty>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocTypes).WithType<AocType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocConfig).WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync();

In [0]:
await DataSource.UpdateAsync<Portfolio>(new[]{ dt1 });
await DataSource.UpdateAsync<Portfolio>(new[]{ dtr1 });
await DataSource.UpdateAsync<GroupOfInsuranceContract>(new[]{ dt11 });
await DataSource.UpdateAsync<GroupOfReinsuranceContract>(new[]{ dtr11 });

In [0]:
await DataSource.UpdateAsync<DataNodeState>(new[]{ dt11State, dtr11State });
await DataSource.UpdateAsync(new[]{ dt11Inter });

In [0]:
await DataSource.UpdateAsync(new[]{ yieldCurve, yieldCurvePrevious });

In [0]:
await Import.FromString(estimateType).WithType<EstimateType>().WithTarget(DataSource).ExecuteAsync();

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await DataSource.UpdateAsync<PartitionByReportingNodeAndPeriod>(new[]{partition, previousPeriodPartition, partitionScenarioMTUP});
await DataSource.UpdateAsync<PartitionByReportingNode>(new[]{partitionReportingNode});

# Check method

In [0]:
static IfrsVariable[] mya;
static IfrsVariable[] myb;
static ImportStorage mystorage;

public async Task<ActivityLog> CheckIfrsVariablesFromImportStorageAsync(string importFormat, IEnumerable<IfrsVariable> inputDataSetForWorkspace, IEnumerable<IfrsVariable> inputDataSetForDataSource, IEnumerable<IfrsVariable> ivsBenchmark, ImportArgs args)
{
    Activity.Start();
    //Prepare Workspace and DataSource
    await Workspace.UpdateAsync<IfrsVariable>(inputDataSetForWorkspace);
    await DataSource.UpdateAsync<IfrsVariable>(inputDataSetForDataSource);
    
    //Set up import storage and test universe
    var testStorage = new ImportStorage(args with {ImportFormat = importFormat}, DataSource, Workspace);
    await testStorage.InitializeAsync();
    var ivs = testStorage.IfrsVariablesByImportIdentity.SelectMany(x => x.Value);
    
    //Clean up Workspace
    //Workspace.Reset();    
    await Workspace.DeleteAsync<IfrsVariable>(inputDataSetForWorkspace);    
    await DataSource.DeleteAsync<IfrsVariable>(inputDataSetForDataSource);    
    
    var errors = new List<string>();
    
    var extraVariablesInStorage = ivs.Except(ivsBenchmark).Select(x => x.ToIdentityString()+" Value: "+x.Value.ToString()).ToArray();
    if(extraVariablesInStorage.Any()) errors.Add( $"IfrsVariables in the storage contain the following items that are not present in the benchmark:\n{string.Join("\n",extraVariablesInStorage)}." );
       
    var extraVariablesInBenchmark = ivsBenchmark.Except(ivs).Select(x => x.ToIdentityString()+" Value: "+x.Value.ToString()).ToArray();
    if(extraVariablesInBenchmark.Any()) errors.Add( $"IfrsVariables in the benchmark contain the following items that are not present in the storage:\n{string.Join("\n",extraVariablesInBenchmark)}." );
        mya = ivs.Except(ivsBenchmark).ToArray();
        myb = ivsBenchmark.Except(ivs).ToArray();
        mystorage = testStorage;
    //find duplicates in storage
    var ivsByIdentityString = ivs.GroupBy(x => x.ToIdentityString()).Where(x => x.Count() > 1).Select(x => x.Key);
    if(ivsByIdentityString.Any()) errors.Add($"IfrsVariables in the storage have duplicated items for:\n{string.Join("\n",ivsByIdentityString)}.");
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

# Test Best Estimate

## Actuals Import 1

There is a previous year, and it's the first time Actuals are imported for the current year.

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };
var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Actuals Import 2

There is a previous year, cash flow were already imported and actuals are imported for the first time.

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.I, EstimateType = EstimateTypes.BE};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.IA, Value = 1500.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 2500.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 5000.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.IA, Value = 1500.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 2500.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 5000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.AA, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, EstimateType = EstimateTypes.AA, Novelty = Novelties.C, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Actuals Reimport

There is a previous year, and it's the second time Actuals are imported for the current year (reimport).

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = 150.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 200.0},
    basicIfrsVariable with {AocType = AocTypes.EOP, Value = 450.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Actuals Reimport with CF

There is a previous year, Cash flows and Actuals were already imported and Actuals are imported again.

In [0]:
var basicAdvanceActualIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};
var basicBeIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.I, EstimateType = EstimateTypes.BE};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicBeIfrsVariable            with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.IA, Value = 1500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.CF, Value = 2500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.EOP, Value = 5000.0},
    basicAdvanceActualIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicAdvanceActualIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = 150.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = 200.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.EOP, Value = 450.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicBeIfrsVariable            with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 1000.0},
    basicBeIfrsVariable            with {AocType = AocTypes.IA, Value = 1500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.CF, Value = 2500.0},
    basicBeIfrsVariable            with {AocType = AocTypes.EOP, Value = 5000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.CF, Value = -15.0},
    basicAdvanceActualIfrsVariable with {AocType = AocTypes.WO, Value = -20.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Actuals Reimport with Primary Secondary Scope

There is a previous year for Primary Scope (GRIC10) 

There is a previous and current year for Secondary Scope (GIC10), 

It's the first time Actuals are imported for the current year

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfReinsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    //Year
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Value = 100.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.CL,  DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 666.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 1000.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.CL,  DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 666.0},
    //Year -1
    basicIfrsVariable with {AocType = AocTypes.IA, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    //From previous Period
    //Actuals
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.DA, Value = 1000.0},
    //Cash flow
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.C, Value = 1000.0},
    
    //From current Period
    //from DB
    basicIfrsVariable with {AocType = AocTypes.IA, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    basicIfrsVariable with {AocType = AocTypes.CF, Novelty = Novelties.I, DataNode = groupOfInsuranceContracts, EstimateType = EstimateTypes.BE, EconomicBasis = EconomicBases.L, Value = 10.0},
    
    //from workspace
    basicIfrsVariable with {AocType = AocTypes.CF, Value = -10.0},
    basicIfrsVariable with {AocType = AocTypes.WO, Value = 15.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Openings Reimport

There are openings for Primary Scope (GIC10) and for Secondary Scope (GRIC10). 

The Openings for the Secondary Scope are re-imported and the Primary Scope must be loaded in the storage. Both are at inception year.

In [0]:
await Workspace.DeleteAsync(await Workspace.Query<DataNodeState>().ToArrayAsync());
await DataSource.DeleteAsync(await DataSource.Query<DataNodeState>().ToArrayAsync());
await Workspace.UpdateAsync(new[]{dt11State with {Year = args.Year, Month = args.Month}, dtr11State with {Year = args.Year, Month = args.Month}});
await DataSource.UpdateAsync(new[]{dt11State with {Year = args.Year, Month = args.Month}, dtr11State with {Year = args.Year, Month = args.Month}});

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.0},
    basicIfrsVariable with {DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 89.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 91.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.0},
    basicIfrsVariable with {DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 91.0},
  };
var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Opening, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, args);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
// Restore workspace
await Workspace.DeleteAsync(await Workspace.Query<DataNodeState>().ToArrayAsync());
await DataSource.DeleteAsync(await DataSource.Query<DataNodeState>().ToArrayAsync());
await DataSource.UpdateAsync(new[]{ dt11State, dtr11State });
await Workspace.UpdateAsync(new[]{ dt11State, dtr11State });

# Test Scenario

## Relaxed query

For the current year Actuals are imported for Best Estimate and a given Scenario. The Relaxed query should return the union of the 2 giving priority to scenario data. 

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CL, Value = 150.0},
  };

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AmountType = AmountTypes.CL, AocType = AocTypes.CL, Value = -99.0},
  };

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.BOP, Value = 100.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AmountType = AmountTypes.CL, AocType = AocTypes.CL, Value = -99.0},
  };

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Secondary scope

The primary scope is a GRIC scenario while the underlying GIC is Best Estimate.

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Value = 100.0},
    basicIfrsVariable with {AocType = AocTypes.CL, Value = 150.0},
};

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AocType = AocTypes.CL, Value = -15.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AmountType = AmountTypes.CL, AocType = AocTypes.CL, Value = -99.0},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.BOP, Value = 100.0},
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.CL, Value = 150.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AocType = AocTypes.CL, Value = -15.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AmountType = AmountTypes.CL, AocType = AocTypes.CL, Value = -99.0},
};

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Scenario Actuals Import 1

Import of Scenario Actuals where DataSource contains openings for scenario and best estimate. Data Node is not at inception year.

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},
    basicIfrsVariable with {AocType = AocTypes.CL, Value = 150.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.7},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.5},
    basicIfrsVariable with {Partition = previousPeriodPartitionScenarioMTUP.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.1}
};

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partition.Id, AocType = AocTypes.BOP, EstimateType = EstimateTypes.DA, Value = 1000.0},  
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 89.5},   
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Scenario Actuals Import 2

Import of Scenario Actuals where DataSource contains openings for scenario and best estimate. Data Node is at inception year.

In [0]:
await Workspace.DeleteAsync(await Workspace.Query<DataNodeState>().ToArrayAsync());
await DataSource.DeleteAsync(await DataSource.Query<DataNodeState>().ToArrayAsync());
await Workspace.UpdateAsync(new[]{dt11State with {Year = args.Year, Month = args.Month}, dtr11State with {Year = args.Year, Month = args.Month}});
await DataSource.UpdateAsync(new[]{dt11State with {Year = args.Year, Month = args.Month}, dtr11State with {Year = args.Year, Month = args.Month}});

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CL, Value = 150.0},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.7},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.5},
    basicIfrsVariable with {Partition = previousPeriodPartitionScenarioMTUP.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.1}
};

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.7},   
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Scenario Actuals Import 3

Import of Scenario Actuals where DataSource contains openings for scenario and best estimate. Only the GRIC is at inception year.

In [0]:
await Workspace.DeleteAsync(await Workspace.Query<DataNodeState>().ToArrayAsync());
await DataSource.DeleteAsync(await DataSource.Query<DataNodeState>().ToArrayAsync());
await Workspace.UpdateAsync(new[]{dt11State, dtr11State with {Year = args.Year, Month = args.Month}});
await DataSource.UpdateAsync(new[]{dt11State, dtr11State with {Year = args.Year, Month = args.Month}});

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {AocType = AocTypes.CL, Value = 150.0},
    basicIfrsVariable with {DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.0},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.7},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, DataNode = groupOfInsuranceContracts, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.5},
    basicIfrsVariable with {Partition = previousPeriodPartitionScenarioMTUP.Id, DataNode = groupOfInsuranceContracts, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.1}
};

var inputDataSetForWorkspace = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 89.5},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, DataNode = groupOfReinsuranceContracts, AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 90.7},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Value = -15.0},
};

var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Actual, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
// Restore workspace
await Workspace.DeleteAsync(await Workspace.Query<DataNodeState>().ToArrayAsync());
await DataSource.DeleteAsync(await DataSource.Query<DataNodeState>().ToArrayAsync());
await DataSource.UpdateAsync(new[]{ dt11State, dtr11State });
await Workspace.UpdateAsync(new[]{ dt11State, dtr11State });

## Scenario CF Import 1

Import of a CF Scenario1 having (ifrsVar incl Opening) BE and Scenario2 in the DataSource.

In [0]:
var basicRawVariable = new RawVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, AocType = AocTypes.CL, EstimateType = EstimateTypes.BE};
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Value = 150.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.5},
    basicIfrsVariable with {Partition = partitionScenarioMTUP2.Id, AocType = AocTypes.CL, Novelty = Novelties.C, Value = 89.1}
};

var inputDataSetForWorkspace = new RawVariable[]{
    basicRawVariable with {Partition = partitionScenarioMTUP.Id, Values = new[]{110.0}},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Value = 150.5},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 89.5},
};

//var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Cashflow, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
//activity

## Scenario CF Import 2

Import of a CF Scenario1 having (ifrsVar incl Opening) BE and Scenario1 in the DataSource.

In [0]:
var basicRawVariable = new RawVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, AocType = AocTypes.CL, EstimateType = EstimateTypes.BE};
var basicIfrsVariable = new IfrsVariable{Partition = partition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = AmountTypes.PR, Novelty = Novelties.C, EstimateType = EstimateTypes.AA};

var inputDataSetForDataSource = new IfrsVariable[]{
    basicIfrsVariable with {Value = 150.0},
    basicIfrsVariable with {Partition = previousPeriodPartition.Id, AocType = AocTypes.EOP, Novelty = Novelties.C, Value = 89.5},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Novelty = Novelties.C, Value = 89.1}
};

var inputDataSetForWorkspace = new RawVariable[]{
    basicRawVariable with {Partition = partitionScenarioMTUP.Id, Values = new[]{110.0}},
};

var ivsBenchmark = new IfrsVariable[]{
    basicIfrsVariable with {Value = 150.5},
    basicIfrsVariable with {AocType = AocTypes.BOP, Novelty = Novelties.I, Value = 89.5},
    basicIfrsVariable with {Partition = partitionScenarioMTUP.Id, AocType = AocTypes.CL, Novelty = Novelties.C, Value = 89.1}
};

//var activity = await CheckIfrsVariablesFromImportStorageAsync(ImportFormats.Cashflow, inputDataSetForWorkspace, inputDataSetForDataSource, ivsBenchmark, argsScenarioMTUP);
//activity

# Data Node Parameter related logic

## Data Preparation

In [0]:
//Define partition
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Cashflow);

var reportingNodePartition = DataSource.Query<PartitionByReportingNode>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode);
if(reportingNodePartition == null) ApplicationMessage.Log(Error.PartitionNotFound);


var currentPartition = DataSource.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year && 
                                                                                    x.Month == args.Month && x.Scenario == args.Scenario);
if(currentPartition == null) ApplicationMessage.Log(Error.PartitionNotFound);


var previousPeriodPartition = Workspace.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year - 1 && 
                                                                                    x.Month == MonthInAYear && x.Scenario == args.Scenario);
if(previousPeriodPartition == null) ApplicationMessage.Log(Error.PartitionNotFound);

In [0]:
var sampleGic = new GroupOfInsuranceContract(){Portfolio = "P1"};
var inputDataGic = new GroupOfInsuranceContract[]{ sampleGic with {SystemName = "Gross1", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                   sampleGic with {SystemName = "Gross2", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                   sampleGic with {SystemName = "Gross3", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                 };

var sampleGric = new GroupOfReinsuranceContract(){Portfolio = "ReP1"};
var inputDataGric = new GroupOfReinsuranceContract[]{ sampleGric with {SystemName = "Reins1", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                      sampleGric with {SystemName = "Reins2", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                      sampleGric with {SystemName = "Reins3", LiabilityType = LiabilityTypes.LRC, ContractualCurrency = "USD", ValuationApproach = "BBA"},
                                                 };
var sampleDnState = new DataNodeState {Partition = reportingNodePartition.Id, Year = args.Year, Month = args.Month, State = State.Active};
var inputDataState = new DataNodeState[]{ sampleDnState with {DataNode = "Gross1"} ,
                                         sampleDnState with {DataNode = "Gross2"} ,
                                         sampleDnState with {DataNode = "Gross3"} ,
                                         sampleDnState with {DataNode = "Reins1"} ,
                                         sampleDnState with {DataNode = "Reins2"} ,
                                         sampleDnState with {DataNode = "Reins3"} ,
                                        };

var samplePreviousDnParam = new InterDataNodeParameter {Partition = reportingNodePartition.Id, ReinsuranceCoverage = 1, Year = args.Year -1, Month = args.Month};
var sampleCurrentDnParam = new InterDataNodeParameter {Partition = reportingNodePartition.Id, ReinsuranceCoverage = 1, Year = args.Year, Month = args.Month};
var inputDataParameter = new InterDataNodeParameter[]{
                                                      samplePreviousDnParam with {DataNode = "Gross3", LinkedDataNode = "Reins3", ReinsuranceCoverage = 1},
                                                      samplePreviousDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins1", ReinsuranceCoverage = 0.1},
                                                      samplePreviousDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.2},
                                                      samplePreviousDnParam with {DataNode = "Gross2", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.3},
                                                      sampleCurrentDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins1", ReinsuranceCoverage = 0.5},
                                                      sampleCurrentDnParam with {DataNode = "Gross1", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.6},
                                                      sampleCurrentDnParam with {DataNode = "Gross2", LinkedDataNode = "Reins2", ReinsuranceCoverage = 0.7},
                                                      sampleCurrentDnParam with {DataNode = "Gross3", LinkedDataNode = "Reins3", ReinsuranceCoverage = 1.0},
                                        };

var sampleRawVar = new RawVariable{AmountType = AmountTypes.PR, AocType = AocTypes.CL, Novelty = Novelties.C, Partition = currentPartition.Id};

In [0]:
public async Task PrepareWorkspaceDataNodes()
{
    await Workspace.UpdateAsync<GroupOfInsuranceContract>(inputDataGic);
    await Workspace.UpdateAsync<GroupOfReinsuranceContract>(inputDataGric);
    await Workspace.UpdateAsync<DataNodeState>(inputDataState);
    await Workspace.UpdateAsync<InterDataNodeParameter>(inputDataParameter);
}

In [0]:
public async Task CleanWorkspaceDataNodes()
{
    await Workspace.DeleteAsync<GroupOfInsuranceContract>(inputDataGic);
    await Workspace.DeleteAsync<GroupOfReinsuranceContract>(inputDataGric);
    await Workspace.DeleteAsync<DataNodeState>(inputDataState);
    await Workspace.DeleteAsync<InterDataNodeParameter>(inputDataParameter);
}

## Get Underlying Gic

In [0]:
public async Task<ActivityLog> CheckGetUnderlyingGicsAsync(RawVariable[] inputDataVariable, Dictionary<string,IEnumerable<string>> underlyingGicBm)
{
    Activity.Start();
    var errors = new List<string>();
    
    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in primaryScopeDn)
    {
        var id = new ImportIdentity(){DataNode = dn};
        if ( underlyingGicBm[dn].Except(testStorage.GetUnderlyingGic(id)).Count() != 0 )
            errors.Add( $"Underlying Gics for DataNode {dn} not matching with BM. Computed: \n{string.Join("\n",testStorage.GetUnderlyingGic(id))} \n Expected : \n{string.Join("\n",underlyingGicBm[dn])}" );
    }
    
    await Workspace.DeleteAsync<RawVariable>(await Workspace.Query<RawVariable>().ToArrayAsync());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var underlyingGicBm = new Dictionary<string,IEnumerable<string>>(){
    {"Reins1",new string[]{"Gross1"}},
    {"Reins2",new string[]{"Gross1","Gross2"}},
};

var activity = await CheckGetUnderlyingGicsAsync(inputDataVariable, underlyingGicBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins2"},
                                         };
var underlyingGicBm = new Dictionary<string,IEnumerable<string>>(){
    {"Reins2",new string[]{"Gross1","Gross2"}},
};

var activity = await CheckGetUnderlyingGicsAsync(inputDataVariable, underlyingGicBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Get Reinsurance Coverage

In [0]:
public async Task<ActivityLog> CheckGetReinsuranceCoverageAsync(RawVariable[] inputDataVariable,  
                                                   Dictionary<(string, string),double> reinsCovBoPBm, 
                                                   Dictionary<(string, string),double> reinsCovEoPBm)
{
    Activity.Start();
    var errors = new List<string>();

    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in primaryScopeDn)
    {
        var aocTypes = new AocStep[]{ new AocStep(AocTypes.BOP, Novelties.I), new AocStep(AocTypes.RCU, Novelties.I) };
        foreach (var aoc in aocTypes)
        {
            var bm = aoc.AocType == AocTypes.BOP ? reinsCovBoPBm : reinsCovEoPBm;
            var id = new ImportIdentity(){DataNode = dn, AocType = aoc.AocType, Novelty = aoc.Novelty};
            
            var computedReinsCov = testStorage.GetUnderlyingGic(id)
                                              .Select(gic => (g: gic, value: testStorage.GetReinsuranceCoverage(id, gic)))
                                              .ToDictionary(x => (dn,x.g), x => x.value);
            
            if( bm.Keys.Where(x => x.Item1 == dn).Except(computedReinsCov.Keys).Count() != 0)
               errors.Add( $"Gric-Gic links not matching with BM for DataNode {dn} and AocType {aoc.AocType}. \n Computed: \n{string.Join("\n",computedReinsCov.Keys)} \n Expected: \n{string.Join("\n",bm.Keys)}" );
            
            foreach (var reinsCov in computedReinsCov)
            {
                var bmKvp = bm.Single(x => x.Key.Item1 == reinsCov.Key.Item1 && x.Key.Item2 == reinsCov.Key.Item2); 
                if( Math.Abs(bmKvp.Value - reinsCov.Value) > Precision )
                    errors.Add( $"{dn}-{reinsCov.Key.Item2} Reinsurance Coverage not matching with BM for AocType {aoc.AocType}: \n Computed: {reinsCov.Value} \n Expected: {bmKvp.Value}");
            }
        }
    }
    
    await Workspace.DeleteAsync<RawVariable>(await Workspace.Query<RawVariable>().ToArrayAsync());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var reinsCovBoPBm = new Dictionary<(string, string),double>(){
    {("Reins1","Gross1"),0.1},
    {("Reins2","Gross1"),0.2},
    {("Reins2","Gross2"),0.3},
};

var reinsCovEoPBm = new Dictionary<(string, string),double>(){
    {("Reins1","Gross1"),0.5},
    {("Reins2","Gross1"),0.6},
    {("Reins2","Gross2"),0.7},
};

var activity = await CheckGetReinsuranceCoverageAsync(inputDataVariable, reinsCovBoPBm, reinsCovEoPBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Primary and Secondary Scope

In [0]:
public async Task<ActivityLog> CheckSecondaryScopeAsync(RawVariable[] inputDataVariable, string[] primaryScopeBm, string[] secondaryScopeBm)
{
    Activity.Start();
    var errors = new List<string>();
    
    await PrepareWorkspaceDataNodes();
    await Workspace.UpdateAsync<RawVariable>(inputDataVariable);
    var testStorage = new ImportStorage(args, DataSource, Workspace);
    await testStorage.InitializeAsync();
    
    var activeDn = (await Workspace.Query<DataNodeState>().ToArrayAsync()).Select(x => x.DataNode);
    
    var primaryScopeDn = testStorage.DataNodesByImportScope[ImportScope.Primary];
    
    foreach (var dn in activeDn)
    {
        //PrimaryScope
        if ( primaryScopeBm.Contains(dn) && !primaryScopeDn.Contains(dn))
             errors.Add( $"DataNode {dn} is not added to the primary scope." );
        if( !primaryScopeBm.Contains(dn) && primaryScopeDn.Contains(dn))
             errors.Add( $"DataNode {dn} is added to the primary scope but should have not." );
        
        //SecondaryScope
        if ( secondaryScopeBm.Contains(dn) && !testStorage.IsSecondaryScope(dn))
             errors.Add( $"DataNode {dn} is not added to the secondary scope." );
        if( !secondaryScopeBm.Contains(dn) && testStorage.IsSecondaryScope(dn))
             errors.Add( $"DataNode {dn} is added to the secondary scope but should have not." );
    }
    
    await Workspace.DeleteAsync<RawVariable>(await Workspace.Query<RawVariable>().ToArrayAsync());
    await CleanWorkspaceDataNodes();
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Reins2"},
                                         };
var primaryScopeBm = new string[]{"Reins1", "Reins2"};
var secondaryScopeBm = new string[]{"Gross1", "Gross2"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           };
var primaryScopeBm = new string[]{"Reins1"};
var secondaryScopeBm = new string[]{"Gross1"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var inputDataVariable = new RawVariable[]{ sampleRawVar with {DataNode = "Reins1"},
                                           sampleRawVar with {DataNode = "Gross1"},
                                           };
var primaryScopeBm = new string[]{"Reins1","Gross1","Reins2"};
var secondaryScopeBm = new string[]{"Gross2"};
var activity = await CheckSecondaryScopeAsync(inputDataVariable, primaryScopeBm, secondaryScopeBm);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);